# Treinamento com interface de alto nível

## Importação das bibliotecas

In [5]:
!pip install openpyxl scikit-learn

In [6]:
# http://pytorch.org/
from os.path import exists

import torch

In [7]:
import argparse
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.optim.lr_scheduler import StepLR
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from torch.utils.data import Dataset, DataLoader

# Importando bibliotecas para leitura do dataset
import pandas as pd

### Dataset escolhido
https://www.kaggle.com/datasets/muratkokludataset/date-fruit-datasets/data

# Dataset do Drive

In [8]:
# conectando com o google drive e adicionando o path para o dataset
from google.colab import drive
drive.mount('/content/drive')

file_path = '/content/drive/My Drive/Dataset/Date_Fruit_Datasets.xlsx'

Mounted at /content/drive


# Montando DF

In [38]:
# Leitura do dataset
df = pd.read_excel(file_path)
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,BERHI
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,BERHI
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,BERHI
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,BERHI
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,BERHI


In [39]:
def encode (df):
    for col in df.columns:
        if df[col].dtype == 'object' or df[col].dtype == 'category':
            le = LabelEncoder()
            df[col] = le.fit_transform(df[col])
    return df
df = encode(df)

In [11]:
df.head()

,AREA,PERIMETER,MAJOR_AXIS,MINOR_AXIS,ECCENTRICITY,EQDIASQ,SOLIDITY,CONVEX_AREA,EXTENT,ASPECT_RATIO,...,KurtosisRR,KurtosisRG,KurtosisRB,EntropyRR,EntropyRG,EntropyRB,ALLdaub4RR,ALLdaub4RG,ALLdaub4RB,Class
0,422163,2378.908,837.8484,645.6693,0.6373,733.1539,0.9947,424428,0.7831,1.2976,...,3.2370,2.9574,4.2287,-59191263232,-50714214400,-39922372608,58.7255,54.9554,47.8400,0
1,338136,2085.144,723.8198,595.2073,0.5690,656.1464,0.9974,339014,0.7795,1.2161,...,2.6228,2.6350,3.1704,-34233065472,-37462601728,-31477794816,50.0259,52.8168,47.8315,0
2,526843,2647.394,940.7379,715.3638,0.6494,819.0222,0.9962,528876,0.7657,1.3150,...,3.7516,3.8611,4.7192,-93948354560,-74738221056,-60311207936,65.4772,59.2860,51.9378,0
3,416063,2351.210,827.9804,645.2988,0.6266,727.8378,0.9948,418255,0.7759,1.2831,...,5.0401,8.6136,8.2618,-32074307584,-32060925952,-29575010304,43.3900,44.1259,41.1882,0
4,347562,2160.354,763.9877,582.8359,0.6465,665.2291,0.9908,350797,0.7569,1.3108,...,2.7016,2.9761,4.4146,-39980974080,-35980042240,-25593278464,52.7743,50.9080,42.6666,0


In [40]:
# Separando X e Y
X = df.iloc[:, :-1].values
y = df.iloc[:, -1].values

In [41]:
X = StandardScaler().fit_transform(X)
X[:5]

array([[ 1.15563972,  0.78394219,  0.60450726,  1.3116427 , -1.12956824,
         1.0757039 ,  0.70862801,  1.10875224,  0.87187257, -0.04679741,
         1.12543323,  1.09122794, -0.06986663, -0.6884426 ,  1.16993442,
         0.24046724,  0.53819492,  0.43558116, -0.02394082, -0.40172428,
        -0.60717863,  0.81076886, -0.69515689, -0.55381039,  0.55548747,
        -0.34968313, -0.57528126,  0.21856505, -1.34281073, -1.26726851,
        -0.82272771,  0.53834033,  0.43559289, -0.02390791],
       [ 0.37169955,  0.06706714, -0.18747301,  0.8697892 , -1.89977082,
         0.43143676,  0.85741044,  0.32337263,  0.80485249, -0.05137327,
         1.68899762,  1.59814193, -0.06497537,  0.01301896,  1.76508448,
         0.51903591, -0.00336621,  0.28410468, -0.02469953, -0.28267776,
        -0.52905251,  0.40278056, -0.34162048, -0.41303366,  0.2574945 ,
        -0.56215422, -0.66140671, -0.29800906, -0.11703018, -0.49322492,
        -0.25342172, -0.00354973,  0.28411298, -0.02469438],
  

In [42]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [43]:
X_train_tensor = torch.tensor(X_train, dtype=torch.float32)
y_train_tensor = torch.tensor(y_train, dtype=torch.long)
X_test_tensor = torch.tensor(X_test, dtype=torch.float32)
y_test_tensor = torch.tensor(y_test, dtype=torch.long)

In [44]:
# pegando o shape da entrada para saber como montar a primeira camada da rede
# como temos 34 colunas, isso vai refletir na primeira camada
X_train_tensor.shape

torch.Size([718, 34])

In [45]:
class CustomDataset(Dataset):
    def __init__(self, X, y):
        self.X = X
        self.y = y

    def __len__(self):
        return len(self.X)

    def __getitem__(self, idx):
        return self.X[idx], self.y[idx]

In [46]:
train_dataset = CustomDataset(X_train_tensor, y_train_tensor)
test_dataset = CustomDataset(X_test_tensor, y_test_tensor)

## Criação da rede

In [47]:
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.fc1 = nn.Linear(34, 1024)
        self.fc2 = nn.Linear(1024, 512)
        self.fc3 = nn.Linear(512, 256)
        self.fc4 = nn.Linear(256, 128)
        self.fc5 = nn.Linear(128, 64)
        self.fc6 = nn.Linear(64, 7)

    def forward(self, x):
        x = x.view(x.shape[0], -1)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = F.relu(self.fc4(x))
        x = F.relu(self.fc5(x))
        x = self.fc6(x)
        output = F.log_softmax(x, dim=1)
        return output

model = Net()

# Predict

In [48]:
model(X_train_tensor)

tensor([[-1.9515, -1.9365, -1.8419,  ..., -2.0052, -1.8623, -2.0049],
        [-1.9479, -1.9345, -1.8483,  ..., -2.0053, -1.8620, -2.0080],
        [-1.9546, -1.9350, -1.8512,  ..., -2.0006, -1.8602, -2.0052],
        ...,
        [-1.9538, -1.9376, -1.8465,  ..., -2.0018, -1.8606, -2.0049],
        [-1.9507, -1.9399, -1.8434,  ..., -2.0059, -1.8616, -2.0033],
        [-1.9550, -1.9375, -1.8441,  ..., -2.0033, -1.8635, -2.0016]],
       grad_fn=<LogSoftmaxBackward0>)

## Treinamento

### Criando o objeto de treinamento

In [49]:
def train(log_interval, dry_run, model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        if batch_idx % log_interval == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
                epoch, batch_idx * len(data), len(train_loader.dataset),
                100. * batch_idx / len(train_loader), loss.item()))
            if dry_run:
                break

In [50]:
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

## Avaliação

In [52]:
use_cuda = torch.cuda.is_available()

torch.manual_seed(1111)

device = torch.device("cuda" if use_cuda else "cpu")

train_kwargs = {'batch_size': 64}
test_kwargs = {'batch_size': 64}
if use_cuda:
    cuda_kwargs = {'num_workers': 1,
                    'pin_memory': True,
                    'shuffle': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))
    ])

train_loader = torch.utils.data.DataLoader(train_dataset,**train_kwargs)
test_loader = torch.utils.data.DataLoader(test_dataset, **test_kwargs)

model = Net().to(device)
optimizer = optim.Adadelta(model.parameters(), lr=1)

epochs = 14
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)

for epoch in range(1, epochs + 1):
    train(10, False, model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

torch.save(model.state_dict(), "date_fruit_cnn.pt")

Train Epoch: 1 [0/718 (0%)]	Loss: 1.934479
Train Epoch: 1 [640/718 (83%)]	Loss: 1.770353

Test set: Average loss: 1.5107, Accuracy: 82/180 (46%)

Train Epoch: 2 [0/718 (0%)]	Loss: 1.494903
Train Epoch: 2 [640/718 (83%)]	Loss: 0.859063

Test set: Average loss: 0.7913, Accuracy: 122/180 (68%)

Train Epoch: 3 [0/718 (0%)]	Loss: 1.054377
Train Epoch: 3 [640/718 (83%)]	Loss: 0.495075

Test set: Average loss: 0.6344, Accuracy: 134/180 (74%)

Train Epoch: 4 [0/718 (0%)]	Loss: 0.613825
Train Epoch: 4 [640/718 (83%)]	Loss: 0.485566

Test set: Average loss: 0.4290, Accuracy: 152/180 (84%)

Train Epoch: 5 [0/718 (0%)]	Loss: 0.488713
Train Epoch: 5 [640/718 (83%)]	Loss: 0.406783

Test set: Average loss: 0.5020, Accuracy: 144/180 (80%)

Train Epoch: 6 [0/718 (0%)]	Loss: 0.525468
Train Epoch: 6 [640/718 (83%)]	Loss: 0.451812

Test set: Average loss: 0.4209, Accuracy: 145/180 (81%)

Train Epoch: 7 [0/718 (0%)]	Loss: 0.279098
Train Epoch: 7 [640/718 (83%)]	Loss: 0.362809

Test set: Average loss: 0.344

### Avaliação sem StandardScaler
Foi executada avaliação com 14 épocas inicialmente atingindo apenas 44% de acurácia e mesmo aumentando para 28 épocas a acurácia, ao invés de melhorar, piorou, diminuindo para 42%.

### Avaliação com StandardScaler
Padronizando os dados previamente com StandardScaler, temos uma melhora considerável no treinamento, com apenas 14 épocas chegamos a uma acurácia de 91%.

### Rede mais complexa
- Deixando a rede mais complexa, adicionando uma camada a mais *fc5*, temos uma melhora na acurácia de 91 para 92%.
- Adicionando mais neurônios (1024) e, consequentemente, adicionando mais uma camada *fc6*, a acurácia continua boa, porém, menor que 91% chegando ao máximo de 87%